# First graph: Importing relevant classes

In [6]:
from langgraph.graph import START, END, StateGraph
from typing_extensions import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.runnables import Runnable
from collections.abc import Sequence 
import config

```python
from langgraph.graph import START, END, StateGraph
```

* **LangGraph** is a library for building workflows with LLMs.
* `START` and `END` → special markers showing where a graph (pipeline) starts and ends.
* `StateGraph` → lets you define a graph of states and transitions (like a flowchart for AI tasks).

---

```python
from typing_extensions import TypedDict
```

* `TypedDict` lets you define a **dictionary with fixed keys and value types**.
* Example:

  ```python
  class MyDict(TypedDict):
      name: str
      age: int
  ```

  → means every `MyDict` must have a `"name"` (string) and `"age"` (integer).

---


```python
from langchain_core.messages import HumanMessage, BaseMessage
```

* `HumanMessage` → represents a message from the **user**.
* `BaseMessage` → the general type for any message (human, AI, system).

---

```python
from langchain_core.runnables import Runnable
```

* `Runnable` → a building block in LangChain.
* It’s like a **function wrapper** that can be chained with others to make a pipeline.

---

```python
from collections.abc import Sequence 
```

* `Sequence` → a generic type meaning “a list-like container” (something you can loop over in order, e.g. list, tuple).

---


In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=config.gemini_api_key,
    temperature=0.7,
    max_tokens=250,
    model_kwargs={"seed": 42}
)

In [8]:
def AskGeminiAI(question) -> str :
    response = llm.invoke(question)
    return response.content